In [17]:
import re

import pandas as pd
from sklearn.cluster import HDBSCAN
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.utils import shuffle

In [ ]:
vpr_df = pd.read_csv('../output/vacancy_professional_roles.csv')
vks_df = pd.read_csv('../output/vacancy_key_skills.csv')

In [4]:
vpr_df

,vacancy_id,professional_role_name
0,108866195,"Программист, разработчик"
1,109005977,"Программист, разработчик"
2,107709799,"Программист, разработчик"
3,108983317,"Программист, разработчик"
4,108127100,"Программист, разработчик"
...,...,...
8484,109683551,Системный администратор
8485,106712051,Системный администратор
8486,108058748,Системный администратор
8487,107789115,Системный администратор


In [5]:
vks_df

,vacancy_id,key_skill_name
0,109005977,PostgreSQL
1,109005977,C#
2,109005977,.NET 6
3,109005977,EF Core
4,109005977,MS SQL
...,...,...
39207,109687310,Администрирование серверов Windows
39208,109687310,VMware
39209,109687310,Информационная безопасность
39210,109687310,Windows Os


In [6]:
df = pd.merge(vpr_df, vks_df, on='vacancy_id')
df

,vacancy_id,professional_role_name,key_skill_name
0,109005977,"Программист, разработчик",PostgreSQL
1,109005977,"Программист, разработчик",C#
2,109005977,"Программист, разработчик",.NET 6
3,109005977,"Программист, разработчик",EF Core
4,109005977,"Программист, разработчик",MS SQL
...,...,...,...
39222,109687310,Системный администратор,Администрирование серверов Windows
39223,109687310,Системный администратор,VMware
39224,109687310,Системный администратор,Информационная безопасность
39225,109687310,Системный администратор,Windows Os


In [7]:
df = shuffle(df, random_state=42)
df

,vacancy_id,professional_role_name,key_skill_name
13295,107315897,Системный инженер,Redis
37544,110614819,Системный администратор,Ремонт ПК
962,108816618,"Программист, разработчик",Reindexer
37455,110913528,Системный администратор,RTPS
17844,109463246,"Программист, разработчик",HTML
...,...,...,...
6265,103911284,Тестировщик,Анализ данных
11284,108519437,DevOps-инженер,DevOps
38158,110309046,Системный администратор,Настройка сетевых подключений
860,107656399,"Программист, разработчик",C#


In [ ]:
# Пример данных
vacancies = pd.DataFrame({'vacancy_id': [1, 2, 3, 4]})
key_skills = pd.DataFrame({
    'key_skill_name': [
        "Kubernetes", "Kubernates", "kubernets",
        "HTML, CSS, JS", "HTML/CSS", "Git", " git ", "GIT"
    ]
})
vacancy_key_skills = pd.DataFrame({
    'vacancy_id': [1, 1, 1, 2, 2, 3, 3, 4],
    'key_skill_name': [
        "Kubernetes", "Kubernates", "kubernets",
        "HTML, CSS, JS", "HTML/CSS", "Git", " git ", "GIT"
    ]
})

# --- 1. Очистка текста ---
def clean_symbols(text):
    """Удаление лишних символов, нормализация пробелов и регистра."""
    text = re.sub(r'[^\w\s]', ' ', text)  # Убираем все символы кроме букв, цифр, пробелов
    text = re.sub(r'\s+', ' ', text).strip()  # Нормализуем пробелы
    return text.lower()  # Приводим к нижнему регистру

key_skills['key_skill_name'] = key_skills['key_skill_name'].apply(clean_symbols)
vacancy_key_skills['key_skill_name'] = vacancy_key_skills['key_skill_name'].apply(clean_symbols)

# --- 2. Разделение навыков, если они перечислены в одной строке ---
def split_skills(skill_string):
    """Разделение перечисленных навыков."""
    return [skill.strip() for skill in re.split(r'[,\;/]', skill_string) if skill.strip()]

# Разворачиваем списки навыков
expanded_skills = vacancy_key_skills['key_skill_name'].apply(split_skills)
vacancy_key_skills = vacancy_key_skills.drop(columns=['key_skill_name']).explode('key_skill_name')

# --- 3. Устранение дубликатов и опечаток с помощью HDBSCAN ---
def cluster_skills(skills):
    """Кластеризация похожих навыков и выбор представителя."""
    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 3))
    X = vectorizer.fit_transform(skills)
    
    clusterer = hdbscan.HDBSCAN(min_cluster_size=2, metric='euclidean')
    labels = clusterer.fit_predict(X)
    
    # Формируем DataFrame с кластерами
    clustered = pd.DataFrame({'original_skill': skills, 'cluster': labels})
    
    # Выбираем представителя для каждого кластера
    def choose_representative(group):
        return group['original_skill'].value_counts().idxmax()
    
    clustered['representative_skill'] = clustered.groupby('cluster').transform(choose_representative)['original_skill']
    
    # Заменяем -1 (шум) оригинальным значением
    clustered['representative_skill'] = clustered.apply(
        lambda row: row['original_skill'] if row['cluster'] == -1 else row['representative_skill'], axis=1
    )
    
    return clustered.set_index('original_skill')['representative_skill'].to_dict()

# Уникальные навыки
unique_skills = vacancy_key_skills['key_skill_name'].unique()

# Кластеризация и замена навыков на представителей
skill_map = cluster_skills(unique_skills)
vacancy_key_skills['key_skill_name'] = vacancy_key_skills['key_skill_name'].map(skill_map)

# --- 4. Объединение данных ---
# Группировка навыков для каждой вакансии
final_data = vacancy_key_skills.groupby('vacancy_id')['key_skill_name'].apply(list).reset_index()

# Итоговый DataFrame
print(final_data)


In [8]:
df = df.groupby('vacancy_id').agg({
    'professional_role_name': lambda x: list(set(x)),
    'key_skill_name': lambda x: list(set(x))
}).reset_index()
df

,vacancy_id,professional_role_name,key_skill_name
0,45163827,[Системный администратор],"[Системное мышление, Linux, Nginx, Администрир..."
1,46823861,"[Программист, разработчик]","[Linux, Python, PostgreSQL, Flask, Docker, Redis]"
2,46954274,[Аналитик],"[BPMN, Работа с требованиями, UML, SOAP, Систе..."
3,54961293,[Системный инженер],"[CI/CD, Ansible, Linux, Nginx, Bash, PostgreSQ..."
4,66766808,"[Программист, разработчик]","[JavaScript, TypeScript, Express, REST API, Ne..."
...,...,...,...
5414,111082906,[Системный администратор],"[Желательно: знания сетевой инфраструктуры, оп..."
5415,111099873,[Системный администратор],"[Настройка ПК, Офисная техника, Настройка сете..."
5416,111112257,[Системный администратор],"[IAC, Ansible, MySQL, Terraform, Linux, Debian..."
5417,111124064,[DevOps-инженер],"[Git, CI/CD, MySQL, Linux, DevOps, Grafana, Do..."


In [9]:
vpr_mlb = MultiLabelBinarizer()
X = vpr_mlb.fit_transform(df['professional_role_name'])
X.shape

(5419, 100)

In [10]:
vpr_mlb.classes_

array(['BI-аналитик, аналитик данных', 'DevOps-инженер', 'Event-менеджер',
       'PR-менеджер', 'SMM-менеджер, контент-менеджер',
       'Агент по недвижимости', 'Администратор', 'Аналитик',
       'Андеррайтер', 'Арт-директор, креативный директор', 'Архитектор',
       'Бизнес-аналитик', 'Бизнес-тренер', 'Брокер', 'Бухгалтер',
       'Генеральный директор, исполнительный директор (CEO)',
       'Главный врач, заведующий отделением', 'Главный инженер проекта',
       'Дата-сайентист', 'Делопроизводитель, архивариус',
       'Дизайнер, художник',
       'Директор по информационным технологиям (CIO)',
       'Директор по маркетингу и PR (CMO)', 'Директор по персоналу (HRD)',
       'Директор юридического департамента (CLO)', 'Диспетчер', 'Другое',
       'Инженер ПНР', 'Инженер ПТО, инженер-сметчик',
       'Инженер по качеству', 'Инженер по эксплуатации',
       'Инженер-конструктор, инженер-проектировщик',
       'Инженер-электроник, инженер-электронщик',
       'Инженер-энергетик, ин

In [12]:
vks_mlb = MultiLabelBinarizer()
y = vks_mlb.fit_transform(df['key_skill_name'])
y.shape

(5419, 4766)

In [13]:
vks_mlb.classes_

array(['#devops #sre #kubernetes #k8s #terraform #python #grafana #SiteReliabilityEngineer',
       '- Опыт работы с СУБД mysql, postgresql, MS Access - Опыт работы с IP телефонией. - Знание OC Linux',
       '.NET', ..., '• Управленческие навыки',
       '• Ценообразование и бюджетирование', '\uf02d Tessa'], dtype=object)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf = RandomForestClassifier(random_state=42)
# clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
# print(clf.score(X_test, y_test))

0.0


In [25]:
metric_scorers = {
    'accuracy': 'accuracy'
}

for metric, scorer in metric_scorers.items():
    score = cross_val_score(clf, X, y, cv=4, scoring=scorer).mean()
    print('-', metric, round(score, 3))

- accuracy 0.001
